In [61]:
%cd /Users/maayanlab/Documents/projects/human-melanoma-signature
import sys
sys.path.append('pipeline/scripts/')
import pandas as pd
import h5py
import numpy as np
import json, operator, math, requests
from geode_jupies import *
import sklearn.metrics.pairwise as smp
from scipy.spatial.distance import cosine
import seaborn as sns; sns.set(color_codes=True)
import RNAseq as RNA
from collections import Counter
import time

/Users/maayanlab/Documents/projects/human-melanoma-signature


In [62]:
%pwd

'/Users/maayanlab/Documents/projects/human-melanoma-signature'

In [63]:
from Melanoma import *

# Code Development for Melanoma Pipeline
## Define input and output files

In [51]:
# infiles S4.4
#infile = 's4-enrichment.dir/melanoma-enrichr-results.txt'
#infile2 = 's4-enrichment.dir/enrichr-results-top.txt'

In [59]:
# infiles
# infile = 's2-signatures.dir/melanoma-geneset-top.json'

In [64]:
#infiles S2.2
sig = 's2-signatures.dir/melanoma-signatures.txt'

In [5]:
# #infiles
# cpm = 's1-expression_data.dir/melanoma-cpm.txt'
# matches = 's2-expression_data.dir/melanoma-idmatches.txt'

In [7]:
# # Infile & outfile
# infiles = ['s1-expression_data.dir/melanoma-cpm.txt']
# outfile = 's1-expression_data.dir/melanoma-IDstatus.txt'

In [8]:
# # Infile & outfile
# infiles = ['rawdata/human_matrix.h5', 'rawdata/m2.csv']
# outfile = 's1-expression_data.dir/melanoma-cpm.txt'

### Test Code

In [67]:
sig_df = pd.read_table(sig).set_index('gene')
sig_df

,1,11,15_a,15_b,15_c,17_a,17_b,17_c,18_a,18_b,...,19_b,19_c,19_d,2_a,2_b,2_c,3,5_a,5_b,5_c
gene,,,,,,,,,,,,,,,,,,,,,
A1BG,2.221160e-04,-0.000741,-0.000528,-0.000293,0.000081,-0.000679,-0.000913,-0.000520,0.000217,-0.000439,...,-0.001003,-7.091145e-04,-0.000192,-0.000019,-0.000432,-0.000729,-0.000202,0.001092,0.000437,1.121692e-04
A2M,6.466985e-03,-0.000725,-0.002951,-0.003329,-0.002101,-0.024287,-0.024964,-0.002382,-0.000219,-0.001720,...,0.000173,1.450300e-04,-0.000058,-0.008256,0.273626,0.135737,0.041137,0.000944,-0.019554,-9.802424e-04
A2ML1,-1.235817e-04,-0.000725,-0.000793,-0.000344,0.000104,-0.000761,-0.000908,-0.000830,0.000205,-0.000298,...,0.000173,-5.797044e-05,0.000139,0.000249,-0.000923,-0.001315,-0.000556,0.001044,0.000681,1.505726e-05
A2MP1,-1.037960e-04,-0.000750,-0.000818,-0.000365,0.000101,-0.000775,-0.000919,-0.000840,0.000208,-0.000287,...,0.000162,-5.557867e-05,0.000130,0.000238,-0.000903,-0.001314,-0.000528,0.001051,0.000619,1.639789e-05
A4GALT,7.294164e-04,-0.000740,-0.000195,-0.001962,-0.000815,-0.000589,-0.000910,-0.000797,0.000213,-0.000504,...,0.000122,-1.564616e-04,0.000164,0.000202,-0.000940,-0.001320,0.002802,0.001034,0.000665,-3.905606e-04
AAAS,-2.887554e-05,-0.000671,0.000764,0.000602,-0.001139,0.000240,0.000240,0.000656,-0.000428,-0.002003,...,0.000154,1.891351e-05,0.000084,0.000367,0.000681,0.000030,0.000057,-0.000884,0.000906,-4.390309e-04
AACS,2.675834e-04,0.005065,-0.000697,-0.000205,-0.000929,-0.000864,-0.000125,-0.000464,-0.000521,0.003434,...,0.001846,7.199311e-04,0.002248,-0.000994,-0.000186,-0.001993,-0.001034,0.000374,-0.000285,4.808830e-04
AACSP1,-9.101854e-05,-0.000709,-0.000794,-0.000353,0.000102,-0.000758,-0.000902,-0.000758,0.000169,-0.000357,...,0.000162,-5.557867e-05,0.000130,0.000544,-0.000930,-0.001318,-0.000554,0.001047,0.000664,1.668024e-05
AADACP1,-1.113141e-04,-0.000780,-0.000779,-0.000362,0.000115,-0.000758,-0.000903,-0.000831,0.000198,-0.000299,...,0.000160,-5.714934e-05,0.000130,0.000273,-0.000924,-0.001313,-0.000557,0.001049,0.000666,1.246117e-05


In [131]:
# Make each signature to a **np.array**
s1=sig_df['1'].values
s19c=sig_df['19_c'].values

# Take vector sum per cluster as a representation of intersection
cluster5_inter= s1 + s19c

sort_idx_cluster5 = np.argsort(cluster5_inter)[::-1]
sorted_genes_cluster5 = sig_df.index[sort_idx_cluster5][:500] #take top 500
sorted_genes_cluster5

Index(['COL1A1', 'COL11A1', 'FTH1', 'ACTG1', 'B2M', 'COL5A1', 'COL6A1', 'MDM2',
       'FLNA', 'RPLP1',
       ...
       'CAPN1', 'PLBD2', 'SNRNP200', 'EPC1', 'CD24', 'CISD1', 'MGAT5B',
       'RASSF4', 'FOXF1', 'MICAL2'],
      dtype='object', name='gene', length=500)

In [139]:
s17a=sig_df['17_a'].values
s17b=sig_df['17_b'].values
s17c=sig_df['17_c'].values
s2a=sig_df['2_a'].values
s2b=sig_df['2_b'].values
s2c=sig_df['2_c'].values
s15a=sig_df['15_a'].values
s15b=sig_df['15_b'].values
s15c=sig_df['15_c'].values
s18a=sig_df['18_a'].values
s18b=sig_df['18_b'].values
s18c=sig_df['18_c'].values
s19a=sig_df['19_a'].values
s19b=sig_df['19_b'].values
s19c=sig_df['19_c'].values
s19d=sig_df['19_d'].values
s1=sig_df['1'].values
s5a=sig_df['5_a'].values
s5b=sig_df['5_b'].values
s5c=sig_df['5_c'].values
s11=sig_df['11'].values
s3=sig_df['3'].values

In [140]:
# Take vector sum per cluster as a representation of intersection
cluster1_inter= s17a + s17b + s17c + s2a + s2b + s2c
cluster2_inter= s15c + s19b + s19d
cluster3_inter= s15a + s15b + s3 + s18b + s11
cluster4_inter= s18c + s5b +s5c
cluster5_inter= s1 + s19c
cluster6_inter= s18a + s5a

In [141]:
# Sort the array in reserve order
sort_idx_1 = np.argsort(cluster1_inter)[::-1]
sort_idx_2 = np.argsort(cluster2_inter)[::-1]
sort_idx_3 = np.argsort(cluster3_inter)[::-1]
sort_idx_4 = np.argsort(cluster4_inter)[::-1]
sort_idx_5 = np.argsort(cluster5_inter)[::-1]
sort_idx_6 = np.argsort(cluster6_inter)[::-1]

In [161]:
# Return an index of top 500 summed genes for this cluster
sorted_genes_1 = sig_df.index[sort_idx_1][:5000]
sorted_genes_2 = sig_df.index[sort_idx_2][:5000]
sorted_genes_3 = sig_df.index[sort_idx_3][:5000]
sorted_genes_4 = sig_df.index[sort_idx_4][:5000]
sorted_genes_5 = sig_df.index[sort_idx_5][:5000]
sorted_genes_6 = sig_df.index[sort_idx_6][:5000]

In [162]:
sorted_genes_1 & sorted_genes_2 & sorted_genes_3 & sorted_genes_4 & sorted_genes_5 & sorted_genes_6


Index(['CDKN1A', 'SGK1', 'LAMC1', 'TNFRSF10B', 'ID3', 'BAG6', 'UHMK1', 'TMED4',
       'STAT3', 'ZNFX1', 'BTG1', 'SSB', 'SRRM2', 'AKIRIN1', 'ATG13', 'IST1',
       'DNAJC5', 'TMED5', 'EFR3A', 'PYCR1', 'AHR', 'RELA', 'EID1', 'KLF9',
       'YTHDF1', 'IDH1', 'SYNGR2', 'TMEM189', 'RNF167', 'SLC35A2', 'TMEM59',
       'RPS24', 'FBXO28', 'SCAMP2', 'CAPS', 'SSH1', 'SEC16A', 'CCNC',
       'TMEM87B', 'ANKRD17', 'BAMBI', 'CYFIP2', 'SENP6', 'STUB1', 'COG5',
       'TBX3', 'RPS6KB2', 'TRIAP1', 'ITGA1', 'ABHD14B', 'OS9', 'RRM2B',
       'TUSC2', 'ZNF791', 'TERF2IP', 'TBC1D20', 'ERP44', 'COG3', 'R3HDM4'],
      dtype='object', name='gene')

In [165]:
# Return an index of top 500 summed genes for this cluster
sorted_genes_1 = sig_df.index[sort_idx_1][:3000]
sorted_genes_2 = sig_df.index[sort_idx_2][:3000]
sorted_genes_3 = sig_df.index[sort_idx_3][:3000]
sorted_genes_4 = sig_df.index[sort_idx_4][:3000]
sorted_genes_5 = sig_df.index[sort_idx_5][:3000]
sorted_genes_6 = sig_df.index[sort_idx_6][:3000]

In [166]:
sorted_genes_1 & sorted_genes_2 & sorted_genes_3 & sorted_genes_4 & sorted_genes_5 & sorted_genes_6


Index(['CDKN1A', 'TNFRSF10B', 'ID3', 'UHMK1', 'STAT3', 'SRRM2', 'AKIRIN1',
       'IST1', 'DNAJC5', 'PYCR1', 'AHR', 'RELA', 'KLF9', 'IDH1', 'SLC35A2',
       'TMEM59', 'RPS24', 'SCAMP2', 'CAPS', 'SSH1', 'SEC16A', 'TMEM87B',
       'ANKRD17'],
      dtype='object', name='gene')

In [234]:
c = Counter()
for i in [enr_res[signature]['top'] for signature in enr_res.keys()]:
    c.update(i['term_name'].iloc[:5])

In [235]:
c.most_common()

[('protein targeting to ER (GO:0045047)', 17),
 ('SRP-dependent cotranslational protein targeting to membrane (GO:0006614)',
  17),
 ('nuclear-transcribed mRNA catabolic process, nonsense-mediated decay (GO:0000184)',
  17),
 ('viral transcription (GO:0019083)', 13),
 ('peptide biosynthetic process (GO:0043043)', 8),
 ('viral process (GO:0016032)', 5),
 ('neutrophil degranulation (GO:0043312)', 4),
 ('extracellular matrix organization (GO:0030198)', 3),
 ('neutrophil mediated immunity (GO:0002446)', 3),
 ('regulated exocytosis (GO:0045055)', 2),
 ('platelet degranulation (GO:0002576)', 2),
 ('rRNA metabolic process (GO:0016072)', 2),
 ('mitochondrial ATP synthesis coupled proton transport (GO:0042776)', 1),
 ('ribosomal small subunit export from nucleus (GO:0000056)', 1),
 ('positive regulation of nitric oxide biosynthetic process (GO:0045429)', 1),
 ('positive regulation of nitric oxide metabolic process (GO:1904407)', 1),
 ('ER to Golgi vesicle-mediated transport (GO:0006888)', 1),
 

In [ ]:
c = Counter()
for i in [results[signature]['top'] for signature in results.keys()]
for i in [results['1']['top']]: 
    c.update(i['Term name'].iloc[:5])
[results[signature]['top'] for signature in results.keys()]

In [176]:
# Create a 3rd dictionary 
from collections import defaultdict
results = defaultdict(dict)
# Iterate through dictionary, create gene lists and save as 'genes'
for signature in dict_top_genes:
    for direction in ('top', 'bottom'):
        genes = dict_top_genes[signature][direction] # value = dict [1st key] [2nd key]
        results[signature][direction] = enrichr_result_to_df(genes,'','GO_Biological_Process_2018')
        #for signature in list(dict_top_genes.keys())[0]:

# freeze default dict for readonly1
results.default_factory = None

In [201]:
# Number of studies to be processed
n = len(study_names)
i = 0
for study in study_names[:]:
    
df_1_top=pd.DataFrame(results['1']['top'])

In [209]:
# if creating a list instead of dictionary
test = []
for signature in dict_top_genes:
    for direction in ('top', 'bottom'):
        genes = dict_top_genes[signature][direction] # value = dict [1st key] [2nd key]
        genes_df = pd.DataFrame(enrichr_result_to_df(genes,'','GO_Biological_Process_2018'))
        #specify names of each geneset (which study and top/bottom they are from)
        test.append([signature, direction, genes_df])

In [ ]:
for study, d1 in D.items():
    for top, v in d1.items():
wanted_keys = dict_df.columns.get_values().tolist() # The keys you want
dict((k, results[k]) for k in wanted_keys if k in results)    